In [2]:
import pandas as pd
import numpy as np

#all 23 student names
students2025 = """Hiro
Toka
Rene
Nene
Yuna H.
Rui
Yuna C.
Kazu
Himi
Kokoro
Aika
Sara
Risako
Hanna
Suzuna
Yuriko
George
Yamato
Daisy
Koriki
Miharu
Taisuke""".split('\n')

#list daily roles
prepared_roles = ['prepared1','prepared2','prepared3']
impromptu_roles = ['impromptu1','impromptu2','impromptu3']
evaluator_roles = ['evaluator1','evaluator2','evaluator3']
aux_roles = [
    "Greeter",
    "Joke Master",
    "Timer",
    "Grammarian and Word of the Day",
    "Ah Counter",
    "Ballot Counter",
    'Sergeant at Arms',
    'Thought of the Day',
    'Stand-in'
]
leadership_roles = [
    "President",
    "Toastmaster",
    "Table Topics Master",
    "General Evaluator"
]

#split students into three leagues of sizes 7, 7, 8
l1, l2, l3 = students2025[:7], students2025[7:14], students2025[14:]

roles = ['prepared','impromptu','evaluator']
roles = [ 
            role + str(num)
            for role in roles
            for num in range(1,4)
]

sched = pd.DataFrame(columns = roles)

sched[['prepared1','prepared2','prepared3']] = np.array([l1, l2, l3[:-1]]).T
sched['impromptu1'] = np.roll(l1,3)
sched['impromptu2'] = np.roll(l2,3)
sched['impromptu3'] = np.roll(l3,3)[:-1]


In [3]:
sched

,prepared1,prepared2,prepared3,impromptu1,impromptu2,impromptu3,evaluator1,evaluator2,evaluator3
0,Hiro,Kazu,Suzuna,Yuna H.,Sara,Koriki,NaN,NaN,NaN
1,Toka,Himi,Yuriko,Rui,Risako,Miharu,NaN,NaN,NaN
2,Rene,Kokoro,George,Yuna C.,Hanna,Taisuke,NaN,NaN,NaN
3,Nene,Aika,Yamato,Hiro,Kazu,Suzuna,NaN,NaN,NaN
4,Yuna H.,Sara,Daisy,Toka,Himi,Yuriko,NaN,NaN,NaN
5,Rui,Risako,Koriki,Rene,Kokoro,George,NaN,NaN,NaN
6,Yuna C.,Hanna,Miharu,Nene,Aika,Yamato,NaN,NaN,NaN


In [4]:
#let's restart with names with a league banner

#rename names in the leagues with banner.
#first, save the originals
raw_names = [l1.copy(), l2.copy(), l3.copy()]

class League:
    """
    League.name
    League.members
    """
    def __init__(self, name, name_list):
        self.name = name
        self.members = np.array(name_list)
        return None

    def __str__(self):
        return self.name
    
    def remove_member(self, member_name:str):
        if member_name not in self.members:
            raise(ValueError)
        self.members.remove(member_name)
        return None
    
    def add_member(self, member_name:str):
        if member_name in self.members:
            raise(ValueError)
        self.members.append(member_name)
        return None

    def rotate(self, shift = 1):
        return np.roll(self.members, shift)

    def add_suffix_to_members(self, suffix:str, spacer = '_', inplace = False):
        return [
            member + spacer + suffix
            for member in self.members
        ]

    def add_prefix_to_members(self, prefix:str, spacer = '_', inplace = False):
        rotated = [
                prefix + spacer + member
                for member in self.members
        ]
        if inplace:
            self.members = rotated
        else:
            return rotated

L1 = League("League 1", l1)
L2 = League("League 2", l2)
L3 = League("League 3", l3)

In [5]:
# construct schedule. The league member lists themselves are as good as any order to start
# -- use these for the prepared speakers.


roles = ['prepared','impromptu','evaluator']
roles = [ role + str(num)
         for role in roles
         for num in range(1,4)
        ]

sched = pd.DataFrame(columns = roles)

prepared_roles = ['prepared1','prepared2','prepared3']
impromptu_roles = ['impromptu1','impromptu2','impromptu3']
evaluator_roles = ['evaluator1','evaluator2','evaluator3']
sched[prepared_roles] = np.array([L1.members, L2.members, L3.members[:-1]]).T

# sched['impromptu1'] = L1.rotate(3)
# sched['impromptu2'] = L2.rotate(5)
# sched['impromptu3'] = L3.rotate(7)

sched[impromptu_roles] = np.array([L1.rotate(1), L2.rotate(3), L3.rotate(5)[:-1]]).T

sched[evaluator_roles] = np.array([L1.rotate(4), L2.rotate(5), L3.rotate(6)[:-1]]).T

sched

,prepared1,prepared2,prepared3,impromptu1,impromptu2,impromptu3,evaluator1,evaluator2,evaluator3
0,Hiro,Kazu,Suzuna,Yuna C.,Sara,Yamato,Nene,Kokoro,George
1,Toka,Himi,Yuriko,Hiro,Risako,Daisy,Yuna H.,Aika,Yamato
2,Rene,Kokoro,George,Toka,Hanna,Koriki,Rui,Sara,Daisy
3,Nene,Aika,Yamato,Rene,Kazu,Miharu,Yuna C.,Risako,Koriki
4,Yuna H.,Sara,Daisy,Nene,Himi,Taisuke,Hiro,Hanna,Miharu
5,Rui,Risako,Koriki,Yuna H.,Kokoro,Suzuna,Toka,Kazu,Taisuke
6,Yuna C.,Hanna,Miharu,Rui,Aika,Yuriko,Rene,Himi,Suzuna


In [6]:
import warnings
warnings.filterwarnings('ignore')

#check for repeats on a single day
#count how many times a student shows up on a given day.
#if they show up more than once, that's bad!
np.any( sched.apply(pd.value_counts, axis = 1) > 1)

False

In [7]:
#but everyone is NaN or 1. Great!




#check for repeats for a single role
role_sum = sched.apply(pd.value_counts, axis = 0)

role_sum[prepared_roles].sum(axis = 1)

Aika       1.0
Daisy      1.0
George     1.0
Hanna      1.0
Himi       1.0
Hiro       1.0
Kazu       1.0
Kokoro     1.0
Koriki     1.0
Miharu     1.0
Nene       1.0
Rene       1.0
Risako     1.0
Rui        1.0
Sara       1.0
Suzuna     1.0
Taisuke    0.0
Toka       1.0
Yamato     1.0
Yuna C.    1.0
Yuna H.    1.0
Yuriko     1.0
dtype: float64

In [8]:
#taisuke is not assigned prepared.

role_sum[impromptu_roles].sum(axis = 1)

Aika       1.0
Daisy      1.0
George     0.0
Hanna      1.0
Himi       1.0
Hiro       1.0
Kazu       1.0
Kokoro     1.0
Koriki     1.0
Miharu     1.0
Nene       1.0
Rene       1.0
Risako     1.0
Rui        1.0
Sara       1.0
Suzuna     1.0
Taisuke    1.0
Toka       1.0
Yamato     1.0
Yuna C.    1.0
Yuna H.    1.0
Yuriko     1.0
dtype: float64

In [9]:
#george is not assigned impromptu


role_sum[evaluator_roles].sum(axis = 1)

Aika       1.0
Daisy      1.0
George     1.0
Hanna      1.0
Himi       1.0
Hiro       1.0
Kazu       1.0
Kokoro     1.0
Koriki     1.0
Miharu     1.0
Nene       1.0
Rene       1.0
Risako     1.0
Rui        1.0
Sara       1.0
Suzuna     1.0
Taisuke    1.0
Toka       1.0
Yamato     1.0
Yuna C.    1.0
Yuna H.    1.0
Yuriko     0.0
dtype: float64

In [11]:
#Yuriko is not assigned evaluator.

sched


,prepared1,prepared2,prepared3,impromptu1,impromptu2,impromptu3,evaluator1,evaluator2,evaluator3
0,Hiro,Kazu,Suzuna,Yuna C.,Sara,Yamato,Nene,Kokoro,George
1,Toka,Himi,Yuriko,Hiro,Risako,Daisy,Yuna H.,Aika,Yamato
2,Rene,Kokoro,George,Toka,Hanna,Koriki,Rui,Sara,Daisy
3,Nene,Aika,Yamato,Rene,Kazu,Miharu,Yuna C.,Risako,Koriki
4,Yuna H.,Sara,Daisy,Nene,Himi,Taisuke,Hiro,Hanna,Miharu
5,Rui,Risako,Koriki,Yuna H.,Kokoro,Suzuna,Toka,Kazu,Taisuke
6,Yuna C.,Hanna,Miharu,Rui,Aika,Yuriko,Rene,Himi,Suzuna


In [12]:
L1.members

array(['Hiro', 'Toka', 'Rene', 'Nene', 'Yuna H.', 'Rui', 'Yuna C.'],
      dtype='<U7')

In [13]:
L2.members

array(['Kazu', 'Himi', 'Kokoro', 'Aika', 'Sara', 'Risako', 'Hanna'],
      dtype='<U6')

In [14]:
L3.members

array(['Suzuna', 'Yuriko', 'George', 'Yamato', 'Daisy', 'Koriki',
       'Miharu', 'Taisuke'], dtype='<U7')

Ok, great, so the given schedule, with a one day with a fourth set of `[Taisuke, George, Yuriko]`.

In [248]:
#### ASSIGNING LEADERSHIP ROLES 

seed = 15

#2032940231 is the Monday schedule. Rejected for number of duplicate leadership roles

#2025, 20329021, 9, 14 runs and fails no initial tests

#runs but fails a test
#2026, 1, 2, 7, 8, 10, 16,18,

#does not run
# 3, 4, 5, 6, 11, 12, 13, 17,20

rng = np.random.default_rng(seed)

#fill in leadership roles

prepared_roles = ['prepared1','prepared2','prepared3']
impromptu_roles = ['impromptu1','impromptu2','impromptu3']
evaluator_roles = ['evaluator1','evaluator2','evaluator3']

aux_roles = [
    "Greeter",
    "Joke Master",
    "Timer",
    "Grammarian and Word of the Day",
    "Ah Counter",
    "Ballot Counter",
    'Sergeant at Arms',
    'Thought of the Day',
    'Stand-in'
]


leadership_roles = [
    "President",
    "Toastmaster",
    "Table Topics Master",
    "General Evaluator"
]

#add roles to schedule and clear them
for role in leadership_roles + aux_roles:
    sched[role] = None

#make a counter for leadership positions
leadership_count = dict(zip(students2025, np.zeros(len(l1+l2+l3),dtype = int)))

#collect all student names
all_the_names = list(L1.members) + list(L2.members) + list(L3.members)

def get_unused_names(series_of_names:pd.Series, all_names = all_the_names):
    return [
        str(name) for name in all_names if name not in series_of_names.values
    ]

def least_leaders(d, level = 0):
    min_val = min(d.values())
    return [k for k, v in d.items() if v <= min_val + level]

# for each day, for each role, get the collection of unused people, and pick out the ones with the lowest leadership role rank.

#for each day,
for day_index, day_sched in sched.iterrows():
    unused_names = get_unused_names(day_sched)
    #for each leadership role,
    for role in leadership_roles:
        
        #pick out the students in unused_names that have the lowest leadership count
        done = 0
        counter = 0
        while not done:
            counter += 1
            if counter > 100000:
                raise(ValueError)
            lowest_unused_names = [
                name for name in np.intersect1d(unused_names,  least_leaders(leadership_count))
            ]
            if len(lowest_unused_names) != 0:
                done = 1
            else:
                pass
                
        #pick out the students who have already done this role
        already_did_this_role = sched[role].unique()
        
        #assign a student from unused names that day and hasn't done it
        random_student = str(rng.choice(lowest_unused_names))
        sched.loc[day_index, role] = random_student
        
        #update the leadership count for that student
        leadership_count[random_student] = leadership_count[random_student] + 1
        
        #update unused names
        unused_names = get_unused_names(day_sched)

print('Done')

#check again for repeat names on a single day.
# are there any students with two roles on the same day?
print(
    f"Students with two roles on the same day:\t{np.any( sched.apply(pd.value_counts, axis = 1) > 1)}"
)

#Have any students done a single leadership role more than once?
print(
    f"Students doing the same leadership role twice:\t{np.any(sched[leadership_roles].apply(pd.value_counts, axis = 0) > 1 )}"
)

#what students have multiple leadership roles?
leadership_vcs = pd.Series(sched[leadership_roles].values.flatten()).value_counts()
leadership_vcs

#show schedules for students with 2 leadership roles.

#get students with 2 leadership roles
two_lead_students = [ student for student in leadership_vcs.index if leadership_vcs[student] == 2]

two_leads = []

for student in two_lead_students:
    these_roles = []
    # print(f"{student}\n------------")
    for day, day_sched in sched.iterrows():
        for role in leadership_roles:
            if sched.loc[day, role] == student:
                these_roles.append(role)
                # print(f"Day {day}\t{role}")
    two_leads.append([student, these_roles])

print(f"Students with two non-general evaluator leadership roles:")
[
    print(entry)for entry in two_leads if 'General Evaluator' not in entry[1]
];

Done
Students with two roles on the same day:	False
Students doing the same leadership role twice:	False
Students with two non-general evaluator leadership roles:
['Hiro', ['Toastmaster', 'President']]
['Kazu', ['President', 'Table Topics Master']]


In [282]:
two_leads

[['Hiro', ['Toastmaster', 'President']],
 ['Risako', ['Table Topics Master', 'General Evaluator']],
 ['Toka', ['General Evaluator', 'Toastmaster']],
 ['Kazu', ['President', 'Table Topics Master']],
 ['George', ['General Evaluator', 'Table Topics Master']],
 ['Yamato', ['President', 'General Evaluator']]]

# Kokoro, Miharu, Risako have two non-general evaluator roles for seed 2032940231. Ask if that's ok!
---

Daisy, Sara, Yamato for seed 9
---

Daisy, George, Yuna C., and Yamato for seed 14
---

Hiro, Kazu (seed 15)
---
day 3 is free!

Yuna H., Koriki (seed 35)
---
no day free for george, t, and yuriko,

Yamato, Kokoro, Yuriko (seed 41)
---

Nene, Koriki, Risako, Hiro (seed 44)
---

Toka, Koriki (seed 67)
---

In [249]:
two_leads

[['Hiro', ['Toastmaster', 'President']],
 ['Risako', ['Table Topics Master', 'General Evaluator']],
 ['Toka', ['General Evaluator', 'Toastmaster']],
 ['Kazu', ['President', 'Table Topics Master']],
 ['George', ['General Evaluator', 'Table Topics Master']],
 ['Yamato', ['President', 'General Evaluator']]]

In [250]:
sched[leadership_roles]

,President,Toastmaster,Table Topics Master,General Evaluator
0,Yuriko,Rui,Risako,Toka
1,Kazu,Koriki,Kokoro,George
2,Nene,Taisuke,Yuna H.,Himi
3,Suzuna,Hiro,Hanna,Daisy
4,Yamato,Rene,Aika,Yuna C.
5,Sara,Miharu,George,Yamato
6,Hiro,Toka,Kazu,Risako


In [251]:
[  thing for thing in sched['General Evaluator'] if thing not in sched[leadership_roles[:-1]].values]

['Himi', 'Daisy', 'Yuna C.']

In [252]:
sched.to_csv('inprog.csv', index = None)

In [253]:
#are there any days without any assignment to `[Taisuke, George, Yuriko]`?

for index, day_sched in sched.iterrows():
    intersection = np.intersect1d(day_sched.dropna().values, ["Taisuke", "George", "Yuriko"])
    if len(intersection) == 0:
        print(f'Day {index} is free!')

Day 3 is free!


In [254]:
#be sure to assign Taisuke, George, Yuriko for Prepared, Impromptu, Evaluator 4 to Day 3.

In [255]:
#fill in the rest
aux_roles

['Greeter',
 'Joke Master',
 'Timer',
 'Grammarian and Word of the Day',
 'Ah Counter',
 'Ballot Counter',
 'Sergeant at Arms',
 'Thought of the Day',
 'Stand-in']

In [256]:
get_unused_names(sched.loc[0])

['Rene',
 'Yuna H.',
 'Himi',
 'Aika',
 'Hanna',
 'Daisy',
 'Koriki',
 'Miharu',
 'Taisuke']

In [257]:
len(_)

9

In [258]:
len(aux_roles)

9

In [259]:
sched.apply(lambda x: len(get_unused_names(x)), axis = 1)

0    9
1    9
2    9
3    9
4    9
5    9
6    9
dtype: int64

In [260]:
unused_names_by_day = sched.apply(lambda x: get_unused_names(x), axis = 1)
unused_names_by_day = unused_names_by_day.to_dict()

sched_working = sched.copy()

seed = 2025
rng = np.random.default_rng(seed)


available_students_by_column = sched[aux_roles].apply(get_unused_names, axis = 0)
#for each day,

for day, day_sched in sched_working.iterrows():
    done_with_day = False
    counter = 0
    available_students_by_day = unused_names_by_day[day].copy()
    #rearrange until it works
    while not done_with_day and counter < 100000:
        if counter%1000 == 0:
            print(counter)
        counter = counter + 1
        rearranged_available_students = available_students_by_day.copy()
        rng.shuffle(rearranged_available_students)
        sched_working.loc[day, aux_roles] = rearranged_available_students
        #if a student shows up more than once for a given role, forget the order
        if np.any(sched_working[aux_roles].apply(pd.value_counts,axis = 0) > 1):
            pass
        else:
            #otherwise, leave the new roles.
            done_with_day = True
            print(f"day {day}: {rearranged_available_students}")

0
day 0: ['Himi', 'Aika', 'Rene', 'Taisuke', 'Yuna H.', 'Hanna', 'Daisy', 'Miharu', 'Koriki']
0
day 1: ['Rene', 'Yuna C.', 'Taisuke', 'Miharu', 'Suzuna', 'Sara', 'Hanna', 'Nene', 'Rui']
0
day 2: ['Suzuna', 'Kazu', 'Aika', 'Yamato', 'Miharu', 'Risako', 'Hiro', 'Yuriko', 'Yuna C.']
0
day 3: ['Taisuke', 'Yuriko', 'Sara', 'Rui', 'Toka', 'Kokoro', 'Yuna H.', 'Himi', 'George']
0
day 4: ['Risako', 'Toka', 'Rui', 'Suzuna', 'Kazu', 'Yuriko', 'George', 'Koriki', 'Kokoro']
0
day 5: ['Hiro', 'Rene', 'Yuriko', 'Hanna', 'Nene', 'Himi', 'Yuna C.', 'Aika', 'Daisy']
0
day 6: ['George', 'Daisy', 'Nene', 'Kokoro', 'Sara', 'Koriki', 'Taisuke', 'Yuna H.', 'Yamato']


In [261]:
rearranged_available_students

['George',
 'Daisy',
 'Nene',
 'Kokoro',
 'Sara',
 'Koriki',
 'Taisuke',
 'Yuna H.',
 'Yamato']

In [262]:
sched_working

,prepared1,prepared2,prepared3,impromptu1,impromptu2,impromptu3,evaluator1,evaluator2,evaluator3,President,Toastmaster,Table Topics Master,General Evaluator,Greeter,Joke Master,Timer,Grammarian and Word of the Day,Ah Counter,Ballot Counter,Sergeant at Arms,Thought of the Day,Stand-in
0,Hiro,Kazu,Suzuna,Yuna C.,Sara,Yamato,Nene,Kokoro,George,Yuriko,Rui,Risako,Toka,Himi,Aika,Rene,Taisuke,Yuna H.,Hanna,Daisy,Miharu,Koriki
1,Toka,Himi,Yuriko,Hiro,Risako,Daisy,Yuna H.,Aika,Yamato,Kazu,Koriki,Kokoro,George,Rene,Yuna C.,Taisuke,Miharu,Suzuna,Sara,Hanna,Nene,Rui
2,Rene,Kokoro,George,Toka,Hanna,Koriki,Rui,Sara,Daisy,Nene,Taisuke,Yuna H.,Himi,Suzuna,Kazu,Aika,Yamato,Miharu,Risako,Hiro,Yuriko,Yuna C.
3,Nene,Aika,Yamato,Rene,Kazu,Miharu,Yuna C.,Risako,Koriki,Suzuna,Hiro,Hanna,Daisy,Taisuke,Yuriko,Sara,Rui,Toka,Kokoro,Yuna H.,Himi,George
4,Yuna H.,Sara,Daisy,Nene,Himi,Taisuke,Hiro,Hanna,Miharu,Yamato,Rene,Aika,Yuna C.,Risako,Toka,Rui,Suzuna,Kazu,Yuriko,George,Koriki,Kokoro
5,Rui,Risako,Koriki,Yuna H.,Kokoro,Suzuna,Toka,Kazu,Taisuke,Sara,Miharu,George,Yamato,Hiro,Rene,Yuriko,Hanna,Nene,Himi,Yuna C.,Aika,Daisy
6,Yuna C.,Hanna,Miharu,Rui,Aika,Yuriko,Rene,Himi,Suzuna,Hiro,Toka,Kazu,Risako,George,Daisy,Nene,Kokoro,Sara,Koriki,Taisuke,Yuna H.,Yamato


In [263]:
#all students do one thing per day, yes?
np.all(sched_working.apply(pd.value_counts, axis = 1) == 1)

True

In [264]:
sched_working.apply(pd.value_counts, axis = 1)

,Aika,Daisy,George,Hanna,Himi,Hiro,Kazu,Kokoro,Koriki,Miharu,Nene,Rene,Risako,Rui,Sara,Suzuna,Taisuke,Toka,Yamato,Yuna C.,Yuna H.,Yuriko
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [265]:
#any students doing more than 1?
np.any(sched_working.apply(pd.value_counts, axis = 0) > 1 )

False

In [280]:
sched_working.index.name = "Day"

sched_working = sched_working.rename(columns = replacer)

#save it
sched_working.to_csv("Toastmasters2025Schedule_seed15.csv")

In [267]:
sched_working

,prepared1,prepared2,prepared3,impromptu1,impromptu2,impromptu3,evaluator1,evaluator2,evaluator3,President,Toastmaster,Table Topics Master,General Evaluator,Greeter,Joke Master,Timer,Grammarian and Word of the Day,Ah Counter,Ballot Counter,Sergeant at Arms,Thought of the Day,Stand-in
Day,,,,,,,,,,,,,,,,,,,,,,
0,Hiro,Kazu,Suzuna,Yuna C.,Sara,Yamato,Nene,Kokoro,George,Yuriko,Rui,Risako,Toka,Himi,Aika,Rene,Taisuke,Yuna H.,Hanna,Daisy,Miharu,Koriki
1,Toka,Himi,Yuriko,Hiro,Risako,Daisy,Yuna H.,Aika,Yamato,Kazu,Koriki,Kokoro,George,Rene,Yuna C.,Taisuke,Miharu,Suzuna,Sara,Hanna,Nene,Rui
2,Rene,Kokoro,George,Toka,Hanna,Koriki,Rui,Sara,Daisy,Nene,Taisuke,Yuna H.,Himi,Suzuna,Kazu,Aika,Yamato,Miharu,Risako,Hiro,Yuriko,Yuna C.
3,Nene,Aika,Yamato,Rene,Kazu,Miharu,Yuna C.,Risako,Koriki,Suzuna,Hiro,Hanna,Daisy,Taisuke,Yuriko,Sara,Rui,Toka,Kokoro,Yuna H.,Himi,George
4,Yuna H.,Sara,Daisy,Nene,Himi,Taisuke,Hiro,Hanna,Miharu,Yamato,Rene,Aika,Yuna C.,Risako,Toka,Rui,Suzuna,Kazu,Yuriko,George,Koriki,Kokoro
5,Rui,Risako,Koriki,Yuna H.,Kokoro,Suzuna,Toka,Kazu,Taisuke,Sara,Miharu,George,Yamato,Hiro,Rene,Yuriko,Hanna,Nene,Himi,Yuna C.,Aika,Daisy
6,Yuna C.,Hanna,Miharu,Rui,Aika,Yuriko,Rene,Himi,Suzuna,Hiro,Toka,Kazu,Risako,George,Daisy,Nene,Kokoro,Sara,Koriki,Taisuke,Yuna H.,Yamato


In [268]:
complete_sched = pd.read_csv("Toastmasters2025Schedule_seed15.csv", index_col = "Day")
complete_sched

,prepared1,prepared2,prepared3,impromptu1,impromptu2,impromptu3,evaluator1,evaluator2,evaluator3,President,Toastmaster,Table Topics Master,General Evaluator,Greeter,Joke Master,Timer,Grammarian and Word of the Day,Ah Counter,Ballot Counter,Sergeant at Arms,Thought of the Day,Stand-in
Day,,,,,,,,,,,,,,,,,,,,,,
0,Hiro,Kazu,Suzuna,Yuna C.,Sara,Yamato,Nene,Kokoro,George,Yuriko,Rui,Risako,Toka,Himi,Aika,Rene,Taisuke,Yuna H.,Hanna,Daisy,Miharu,Koriki
1,Toka,Himi,Yuriko,Hiro,Risako,Daisy,Yuna H.,Aika,Yamato,Kazu,Koriki,Kokoro,George,Rene,Yuna C.,Taisuke,Miharu,Suzuna,Sara,Hanna,Nene,Rui
2,Rene,Kokoro,George,Toka,Hanna,Koriki,Rui,Sara,Daisy,Nene,Taisuke,Yuna H.,Himi,Suzuna,Kazu,Aika,Yamato,Miharu,Risako,Hiro,Yuriko,Yuna C.
3,Nene,Aika,Yamato,Rene,Kazu,Miharu,Yuna C.,Risako,Koriki,Suzuna,Hiro,Hanna,Daisy,Taisuke,Yuriko,Sara,Rui,Toka,Kokoro,Yuna H.,Himi,George
4,Yuna H.,Sara,Daisy,Nene,Himi,Taisuke,Hiro,Hanna,Miharu,Yamato,Rene,Aika,Yuna C.,Risako,Toka,Rui,Suzuna,Kazu,Yuriko,George,Koriki,Kokoro
5,Rui,Risako,Koriki,Yuna H.,Kokoro,Suzuna,Toka,Kazu,Taisuke,Sara,Miharu,George,Yamato,Hiro,Rene,Yuriko,Hanna,Nene,Himi,Yuna C.,Aika,Daisy
6,Yuna C.,Hanna,Miharu,Rui,Aika,Yuriko,Rene,Himi,Suzuna,Hiro,Toka,Kazu,Risako,George,Daisy,Nene,Kokoro,Sara,Koriki,Taisuke,Yuna H.,Yamato


In [269]:
complete_sched.loc[1] == "Hiro"

prepared1                         False
prepared2                         False
prepared3                         False
impromptu1                         True
impromptu2                        False
impromptu3                        False
evaluator1                        False
evaluator2                        False
evaluator3                        False
President                         False
Toastmaster                       False
Table Topics Master               False
General Evaluator                 False
Greeter                           False
Joke Master                       False
Timer                             False
Grammarian and Word of the Day    False
Ah Counter                        False
Ballot Counter                    False
Sergeant at Arms                  False
Thought of the Day                False
Stand-in                          False
Name: 1, dtype: bool

In [270]:
inverted_sched = pd.DataFrame(index = range(1,8), columns = students2025)

for student in students2025:
    print(f"{student}\n---\nDay\tRole")
    for day, day_sched in complete_sched.iterrows():
        for column in complete_sched.columns:
            if day_sched[column] == student:
                print(f"{day},\t{column}")
                inverted_sched.loc[day, student] = column
    print("-"*50)



Hiro
---
Day	Role
0,	prepared1
1,	impromptu1
2,	Sergeant at Arms
3,	Toastmaster
4,	evaluator1
5,	Greeter
6,	President
--------------------------------------------------
Toka
---
Day	Role
0,	General Evaluator
1,	prepared1
2,	impromptu1
3,	Ah Counter
4,	Joke Master
5,	evaluator1
6,	Toastmaster
--------------------------------------------------
Rene
---
Day	Role
0,	Timer
1,	Greeter
2,	prepared1
3,	impromptu1
4,	Toastmaster
5,	Joke Master
6,	evaluator1
--------------------------------------------------
Nene
---
Day	Role
0,	evaluator1
1,	Thought of the Day
2,	President
3,	prepared1
4,	impromptu1
5,	Ah Counter
6,	Timer
--------------------------------------------------
Yuna H.
---
Day	Role
0,	Ah Counter
1,	evaluator1
2,	Table Topics Master
3,	Sergeant at Arms
4,	prepared1
5,	impromptu1
6,	Thought of the Day
--------------------------------------------------
Rui
---
Day	Role
0,	Toastmaster
1,	Stand-in
2,	evaluator1
3,	Grammarian and Word of the Day
4,	Timer
5,	prepared1
6,	impromptu1
--------

In [275]:
replacer = {
    "impromptu1":"Impromptu Speaker 1",
    "impromptu2":"Impromptu Speaker 2",
    "impromptu3":"Impromptu Speaker 3",
    "prepared1":"Prepared Speaker 1",
    "prepared2":"Prepared Speaker 2",
    "prepared3":"Prepared Speaker 3",
    "evaluator1":"Evaluator 1",
    "evaluator2":"Evaluator 2",
    "evaluator3":"Evaluator 3",
}

inverted_sched = inverted_sched.replace(replacer)

In [279]:
inverted_sched.index.name = "Day"
#rename index group and one index value
inverted_sched = inverted_sched.drop(7).rename(index={0:7})
inverted_sched.to_csv('Toastmasters2025seed15_by_name.csv')

,Hiro,Toka,Rene,Nene,Yuna H.,Rui,Yuna C.,Kazu,Himi,Kokoro,Aika,Sara,Risako,Hanna,Suzuna,Yuriko,George,Yamato,Daisy,Koriki,Miharu,Taisuke
Day,,,,,,,,,,,,,,,,,,,,,,
1,Impromptu Speaker 1,Prepared Speaker 1,Greeter,Thought of the Day,Evaluator 1,Stand-in,Joke Master,President,Prepared Speaker 2,Table Topics Master,Evaluator 2,Ballot Counter,Impromptu Speaker 2,Sergeant at Arms,Ah Counter,Prepared Speaker 3,General Evaluator,Evaluator 3,Impromptu Speaker 3,Toastmaster,Grammarian and Word of the Day,Timer
2,Sergeant at Arms,Impromptu Speaker 1,Prepared Speaker 1,President,Table Topics Master,Evaluator 1,Stand-in,Joke Master,General Evaluator,Prepared Speaker 2,Timer,Evaluator 2,Ballot Counter,Impromptu Speaker 2,Greeter,Thought of the Day,Prepared Speaker 3,Grammarian and Word of the Day,Evaluator 3,Impromptu Speaker 3,Ah Counter,Toastmaster
3,Toastmaster,Ah Counter,Impromptu Speaker 1,Prepared Speaker 1,Sergeant at Arms,Grammarian and Word of the Day,Evaluator 1,Impromptu Speaker 2,Thought of the Day,Ballot Counter,Prepared Speaker 2,Timer,Evaluator 2,Table Topics Master,President,Joke Master,Stand-in,Prepared Speaker 3,General Evaluator,Evaluator 3,Impromptu Speaker 3,Greeter
4,Evaluator 1,Joke Master,Toastmaster,Impromptu Speaker 1,Prepared Speaker 1,Timer,General Evaluator,Ah Counter,Impromptu Speaker 2,Stand-in,Table Topics Master,Prepared Speaker 2,Greeter,Evaluator 2,Grammarian and Word of the Day,Ballot Counter,Sergeant at Arms,President,Prepared Speaker 3,Thought of the Day,Evaluator 3,Impromptu Speaker 3
5,Greeter,Evaluator 1,Joke Master,Ah Counter,Impromptu Speaker 1,Prepared Speaker 1,Sergeant at Arms,Evaluator 2,Ballot Counter,Impromptu Speaker 2,Thought of the Day,President,Prepared Speaker 2,Grammarian and Word of the Day,Impromptu Speaker 3,Timer,Table Topics Master,General Evaluator,Stand-in,Prepared Speaker 3,Toastmaster,Evaluator 3
6,President,Toastmaster,Evaluator 1,Timer,Thought of the Day,Impromptu Speaker 1,Prepared Speaker 1,Table Topics Master,Evaluator 2,Grammarian and Word of the Day,Impromptu Speaker 2,Ah Counter,General Evaluator,Prepared Speaker 2,Evaluator 3,Impromptu Speaker 3,Greeter,Stand-in,Joke Master,Ballot Counter,Prepared Speaker 3,Sergeant at Arms
7,Prepared Speaker 1,General Evaluator,Timer,Evaluator 1,Ah Counter,Toastmaster,Impromptu Speaker 1,Prepared Speaker 2,Greeter,Evaluator 2,Joke Master,Impromptu Speaker 2,Table Topics Master,Ballot Counter,Prepared Speaker 3,President,Evaluator 3,Impromptu Speaker 3,Sergeant at Arms,Stand-in,Thought of the Day,Grammarian and Word of the Day


In [281]:
complete_sched

,prepared1,prepared2,prepared3,impromptu1,impromptu2,impromptu3,evaluator1,evaluator2,evaluator3,President,Toastmaster,Table Topics Master,General Evaluator,Greeter,Joke Master,Timer,Grammarian and Word of the Day,Ah Counter,Ballot Counter,Sergeant at Arms,Thought of the Day,Stand-in
Day,,,,,,,,,,,,,,,,,,,,,,
0,Hiro,Kazu,Suzuna,Yuna C.,Sara,Yamato,Nene,Kokoro,George,Yuriko,Rui,Risako,Toka,Himi,Aika,Rene,Taisuke,Yuna H.,Hanna,Daisy,Miharu,Koriki
1,Toka,Himi,Yuriko,Hiro,Risako,Daisy,Yuna H.,Aika,Yamato,Kazu,Koriki,Kokoro,George,Rene,Yuna C.,Taisuke,Miharu,Suzuna,Sara,Hanna,Nene,Rui
2,Rene,Kokoro,George,Toka,Hanna,Koriki,Rui,Sara,Daisy,Nene,Taisuke,Yuna H.,Himi,Suzuna,Kazu,Aika,Yamato,Miharu,Risako,Hiro,Yuriko,Yuna C.
3,Nene,Aika,Yamato,Rene,Kazu,Miharu,Yuna C.,Risako,Koriki,Suzuna,Hiro,Hanna,Daisy,Taisuke,Yuriko,Sara,Rui,Toka,Kokoro,Yuna H.,Himi,George
4,Yuna H.,Sara,Daisy,Nene,Himi,Taisuke,Hiro,Hanna,Miharu,Yamato,Rene,Aika,Yuna C.,Risako,Toka,Rui,Suzuna,Kazu,Yuriko,George,Koriki,Kokoro
5,Rui,Risako,Koriki,Yuna H.,Kokoro,Suzuna,Toka,Kazu,Taisuke,Sara,Miharu,George,Yamato,Hiro,Rene,Yuriko,Hanna,Nene,Himi,Yuna C.,Aika,Daisy
6,Yuna C.,Hanna,Miharu,Rui,Aika,Yuriko,Rene,Himi,Suzuna,Hiro,Toka,Kazu,Risako,George,Daisy,Nene,Kokoro,Sara,Koriki,Taisuke,Yuna H.,Yamato
